In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
%matplotlib inline

In [4]:
def acc_plot(history, name):
    plt.figure(figsize=(10,6))
    plt.plot(history['accuracy'], label='training accuracy')
    plt.plot(history['loss'], label='training loss')    
    plt.plot(history['val_accuracy'], label='validation accuracy')
    plt.plot(history['val_loss'], label='validation loss')
    plt.title(name + ' Accuracy and Loss')
    plt.xlabel('epochs')
    plt.legend()
#     plt.savefig(name +'_Result.png', bbox_inches='tight', dpi=300)
    return plt.show()

def gen_csv(model, name):
    result = dfTest[['Id']].copy()
    result['Label'] = model.predict_classes(MinMaxScaler().fit_transform(dfTest.iloc[:,1:]).reshape((-1, 28, 28, 1)))
    result.to_csv('result' + name + '.csv', encoding='utf-8', index=False)

In [5]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
dfTest = pd.read_csv('testX.csv')
X_train, X_val, y_train, y_val = train_test_split(MinMaxScaler().fit_transform(df.iloc[:,1:]).reshape((-1, 28, 28, 1)), to_categorical(df.iloc[:,0], num_classes=5), test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(MinMaxScaler().fit_transform(df.iloc[:,1:]).reshape((-1, 28, 28, 1)), to_categorical(df.iloc[:,0], num_classes=5), test_size=0.2, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.67, random_state = 42)

In [ ]:
CNN1 = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])
CNN1.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
historyCNN1 = CNN1.fit(X_train, y_train, epochs = 30, validation_data = (X_val, y_val))
acc_plot(historyCNN1.history, 'CNN1')
gen_csv(CNN1, 'CNN1')

Train on 48000 samples, validate on 12000 samples
Epoch 1/30
48000/48000 [==============================] - 8s 166us/sample - loss: 0.5811 - accuracy: 0.7552 - val_loss: 0.4182 - val_accuracy: 0.8266
Epoch 2/30
48000/48000 [==============================] - 7s 152us/sample - loss: 0.4314 - accuracy: 0.8243 - val_loss: 0.3643 - val_accuracy: 0.8585
Epoch 3/30
48000/48000 [==============================] - 7s 154us/sample - loss: 0.3832 - accuracy: 0.8433 - val_loss: 0.3693 - val_accuracy: 0.8502
Epoch 4/30
48000/48000 [==============================] - 7s 156us/sample - loss: 0.3530 - accuracy: 0.8571 - val_loss: 0.3135 - val_accuracy: 0.8752
Epoch 5/30
48000/48000 [==============================] - 7s 156us/sample - loss: 0.3316 - accuracy: 0.8658 - val_loss: 0.3337 - val_accuracy: 0.8602
Epoch 6/30
48000/48000 [==============================] - 7s 155us/sample - loss: 0.3141 - accuracy: 0.8715 - val_loss: 0.3302 - val_accuracy: 0.8654
Epoch 7/30
48000/48000 [==========================

In [10]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss="categorical_crossentropy", metrics=["accuracy"])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [11]:
model.fit(X_train, y_train, epochs=40, batch_size=128)
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples
Epoch 1/40
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 24 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
48000/48000 [==============================] - 42s 878us/sample - loss: 0.8456 - accuracy: 0.6759
Epoch 2/40
48000/48000 [==============================] - 12s 240us/sample - loss: 0.6095 - accuracy: 0.7552
Epoch 3/40
48000/48000 [==============================] - 11s 230us/sample - loss: 0.5439 - accuracy: 0.7822
Epoch 4/40
48000/48000 [==============================] - 10s 215us/sample - loss: 0.4942 - accuracy: 0.8028
Epoch 5/40
48000/48000 [==============================] - 10s 211us/sample - loss: 0.4595 - accuracy: 0.8204
Epoch 6/40
48000/48000 [==============================] - 10s 217us/sample - loss: 0.4374 - accuracy: 0.8287
Epoch 7/40
48000/48

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 8, zoom_range = 0.1, shear_range = 0.3, width_shift_range=0.08, height_shift_range=0.08, vertical_flip=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(datagen.flow(X_train, y_train, batch_size = 128), epochs = 40, validation_data = (X_test, y_test), steps_per_epoch=X_train.shape[0] // 128, callbacks = [reduce_lr])
score = model.evaluate(X_test, y_test)

print('Loss: {:.4f}'.format(score[0]))
print('Accuracy: {:.4f}'.format(score[1]))

  ...
    to  
  ['...']
Train for 375 steps, validate on 12000 samples
Epoch 1/40
375/375 [==============================] - 23s 63ms/step - loss: 0.6649 - accuracy: 0.7512 - val_loss: 0.3261 - val_accuracy: 0.8732
Epoch 2/40
375/375 [==============================] - 21s 55ms/step - loss: 0.5207 - accuracy: 0.7897 - val_loss: 0.3525 - val_accuracy: 0.8547
Epoch 3/40
375/375 [==============================] - 22s 58ms/step - loss: 0.4928 - accuracy: 0.8031 - val_loss: 0.3338 - val_accuracy: 0.8637
Epoch 4/40
375/375 [==============================] - 20s 54ms/step - loss: 0.4783 - accuracy: 0.8084 - val_loss: 0.4013 - val_accuracy: 0.8298
Epoch 5/40
375/375 [==============================] - 21s 56ms/step - loss: 0.4631 - accuracy: 0.8143 - val_loss: 0.6324 - val_accuracy: 0.7389
Epoch 6/40
375/375 [==============================] - 21s 56ms/step - loss: 0.4547 - accuracy: 0.8170 - val_loss: 0.3197 - val_accuracy: 0.8668
Epoch 7/40
375/375 [==============================] - 21s 57ms/s

In [3]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss="categorical_crossentropy", metrics=["accuracy"])

In [5]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data = (X_test, y_test), callbacks = [reduce_lr])
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 8s 174us/sample - loss: 0.3455 - accuracy: 0.8625 - val_loss: 0.2962 - val_accuracy: 0.8742
Epoch 2/50
48000/48000 [==============================] - 8s 165us/sample - loss: 0.3384 - accuracy: 0.8649 - val_loss: 0.4136 - val_accuracy: 0.8307
Epoch 3/50
48000/48000 [==============================] - 8s 163us/sample - loss: 0.3269 - accuracy: 0.8704 - val_loss: 0.2907 - val_accuracy: 0.8819
Epoch 4/50
48000/48000 [==============================] - 8s 165us/sample - loss: 0.3165 - accuracy: 0.8746 - val_loss: 0.2683 - val_accuracy: 0.8883
Epoch 5/50
48000/48000 [==============================] - 8s 162us/sample - loss: 0.3184 - accuracy: 0.8736 - val_loss: 0.2603 - val_accuracy: 0.8951
Epoch 6/50
48000/48000 [==============================] - 8s 166us/sample - loss: 0.3105 - accuracy: 0.8750 - val_loss: 0.4409 - val_accuracy: 0.8234
Epoch 7/50
48000/48000 [==========================

In [4]:
model3 = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
#     BatchNormalization(),
#     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Dropout(0.5),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Dropout(0.5),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(3, 3)),
#     BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
#     BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
#     BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model3.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model3.fit(X_train, y_train, epochs=50, batch_size=128, validation_data = (X_val, y_val))
loss, acc = model3.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples, validate on 3959 samples
Epoch 1/50
48000/48000 [==============================] - 7s 154us/sample - loss: 0.8056 - accuracy: 0.6500 - val_loss: 0.4770 - val_accuracy: 0.8035
Epoch 2/50
48000/48000 [==============================] - 4s 92us/sample - loss: 0.5500 - accuracy: 0.7766 - val_loss: 0.4159 - val_accuracy: 0.8272
Epoch 3/50
48000/48000 [==============================] - 5s 94us/sample - loss: 0.4891 - accuracy: 0.8031 - val_loss: 0.4310 - val_accuracy: 0.8126
Epoch 4/50
48000/48000 [==============================] - 5s 97us/sample - loss: 0.4538 - accuracy: 0.8152 - val_loss: 0.3531 - val_accuracy: 0.8588
Epoch 5/50
48000/48000 [==============================] - 4s 93us/sample - loss: 0.4248 - accuracy: 0.8275 - val_loss: 0.3564 - val_accuracy: 0.8535
Epoch 6/50
48000/48000 [==============================] - 5s 95us/sample - loss: 0.4015 - accuracy: 0.8397 - val_loss: 0.3329 - val_accuracy: 0.8702
Epoch 7/50
48000/48000 [==============================] 

In [9]:
model4 = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
#     MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
#     MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.5),
    
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),

#     Dropout(0.5),
    Flatten(),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(5, activation='softmax')
])
model4.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model4.fit(X_train, y_train, epochs=50, batch_size=128, validation_data = (X_val, y_val))
loss, acc = model4.evaluate(X_test, y_test)
print('Accuracy: %.3f' % acc)

Train on 48000 samples, validate on 3959 samples
Epoch 1/50
48000/48000 [==============================] - 14s 300us/sample - loss: 0.9223 - accuracy: 0.6424 - val_loss: 39.5883 - val_accuracy: 0.2008
Epoch 2/50
48000/48000 [==============================] - 13s 262us/sample - loss: 0.6906 - accuracy: 0.7178 - val_loss: 0.8555 - val_accuracy: 0.6449
Epoch 3/50
48000/48000 [==============================] - 13s 261us/sample - loss: 0.5824 - accuracy: 0.7648 - val_loss: 1.0548 - val_accuracy: 0.5580
Epoch 4/50
48000/48000 [==============================] - 13s 264us/sample - loss: 0.5441 - accuracy: 0.7801 - val_loss: 0.4859 - val_accuracy: 0.8141
Epoch 5/50
48000/48000 [==============================] - 13s 264us/sample - loss: 0.5152 - accuracy: 0.7928 - val_loss: 0.8056 - val_accuracy: 0.7148
Epoch 6/50
48000/48000 [==============================] - 13s 264us/sample - loss: 0.4929 - accuracy: 0.7999 - val_loss: 0.4084 - val_accuracy: 0.8340
Epoch 7/50
48000/48000 [====================